## Automate Event Planning

In [1]:
#!pip install -U crewai crewai_tools langchain_community

In [2]:
from crewai import Agent, Task, Crew
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

# from langchain_core.messages import HumanMessage
# message = HumanMessage(
#     content="Translate this sentence from English to French. I love programming."
# )
# r = default_llm.invoke([message])
# print(r)

## Tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Agents

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Tasks

In [5]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "The market is crazy about Gen AI hype and few peple consider "
        "that the valuation of {stock_selection} is pure bubble, "
        "considering current market conditions "
        "evaluate the most likely timelines for possinle siginificant decline "
        "in {stock_selection} stock price and suggest best shorting strategy - "
        "when and how much money to bet at trading platform (with leverage), "
        "short operation has 0.004% overnight fee."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate whether the curent situation can qualified as financial bubble "
        "the probability of rapid fall of {stock_selection} stock. "
        "Provide a detailed analysis of potential risks "
        "and execution strategy as proposed by Trade Advisor, "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


## Crew

- The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [6]:
from crewai import Crew, Process

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
           
	# embedder={  
    #     "provider": "azure_openai",
    #     "config": {
    #         "model": "text-embedding-ada-002", 
    #         "deployment_name": "text-embedding-ada-002",
    #         "api_key": os.environ.get("AZURE_OPENAI_KEY")
    #     }
    # },
    
    manager_llm=default_llm,
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

In [7]:
from datetime import datetime

# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'Nvidia',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'current_date': datetime.now().strftime("%Y-%m-%d"),
    'current_date': 'Current date',
}

### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (Nvidia). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
To complete the task of continuously monitoring and analyzing market data for Nvidia, identifying trends, and predicting market movements using statistical modeling and machine learning, I need to involve the appropriate team members. This task will likely require contributions from the Data Analyst for data collection and initial analysis, the Trading Strategy Developer for creating and optimizing models, and the Risk Advisor for identifying potential risks and threats in the market.

First, I need to gather data and perform an initial analysis. This will be the responsibility of the Data Analyst.

Thought: I will delegate the task of gathering and performing an initial analysis of Nvidia's market data to the D

In [8]:
from IPython.display import Markdown
Markdown(result)

### Comprehensive Risk Analysis Report for Shorting Nvidia's Stock

#### Introduction
This report evaluates whether the current situation qualifies as a financial bubble and assesses the probability of a rapid fall in Nvidia's stock. The analysis considers the detailed insights provided by the Data Analyst, Trading Strategy Developer, Trade Advisor, and Risk Advisor. 

#### Detailed Execution Plan for Shorting Nvidia's Stock

Based on the insights provided by the Data Analyst regarding Nvidia's stock performance over the past six months, and the detailed strategies developed by the Trading Strategy Developer and the Risk Advisor, here is a comprehensive execution plan for shorting Nvidia's stock.

#### Strategy Overview
- **Initiation**: Initiate a short position after a minor rally.
- **Leverage**: Use 5x leverage.
- **Risk**: Risk 2% of the account value per trade.
- **Stop Loss**: Set stop losses to manage risk.
- **Take Profit**: Take profit at identified support levels.
- **Fees**: Factor in a 0.004% overnight fee.
- **Monitoring**: Regularly monitor and adjust positions based on market sentiment and performance.

#### Recent Performance and Volatility Trends
- **6-Month Performance**: Nvidia's stock has shown significant volatility with a performance increase of approximately 159.50%.
- **Recent Highs**: The stock reached a high of $140.76 on 06/20/24.
- **Recent Lows**: The stock had a low of $47.32 on 01/03/24.

#### Current Support and Resistance Levels
- **Resistance Levels**:
  - 1st Resistance Point: $137.85
  - 2nd Resistance Point: $144.93
  - 3rd Resistance Point: $149.09
- **Support Levels**:
  - 1st Support Level: $126.61
  - 2nd Support Level: $122.45
  - 3rd Support Level: $115.37
- **Current Price**: Approximately $126.06

#### Trade Execution Steps
1. **Identify Entry Point**:
   - Wait for a minor rally close to the first resistance point ($137.85).
   - Ensure market conditions are favorable and sentiment is neutral to bearish.

2. **Position Sizing**:
   - Calculate the amount to short based on 2% of total account value.
   - Example: For an account value of $100,000, risk $2,000 per trade.

3. **Leverage Application**:
   - Use 5x leverage to increase exposure.
   - Example: With $2,000 risked, the leveraged position size would be $10,000.

4. **Set Stop Loss**:
   - Place a stop loss slightly above the entry point to limit potential losses.
   - Adjust stop loss based on market conditions and volatility.

5. **Set Take Profit**:
   - Set take profit levels at identified support levels.
   - Example: Initial take profit at the first support level ($126.61).

6. **Monitor Overnight Fees**:
   - Calculate the impact of the 0.004% overnight fee on the leveraged position.
   - Example: For a $10,000 position, the overnight fee would be $0.40.

7. **Regular Monitoring and Adjustment**:
   - Monitor the position regularly for any changes in market sentiment or performance.
   - Adjust stop losses and take profit levels as necessary to lock in gains or minimize losses.

8. **Risk Mitigation**:
   - Reduce leverage if market conditions become highly volatile or unfavorable.
   - Tighten stop losses to protect against sudden adverse movements.

#### Risk Management
- **Leverage Reduction**: In case of increased volatility, reduce leverage to lower exposure.
- **Tightening Stop Losses**: Adjust stop loss levels to minimize potential losses.
- **Market Sentiment**: Monitor overall market sentiment and news related to Nvidia to anticipate potential impacts on stock performance.

By following this execution plan, you can effectively short Nvidia's stock while managing risk and maximizing potential returns. Regular monitoring and adjustments are crucial to adapt to changing market conditions.

### Detailed Analysis of Potential Risks and Execution Strategy

#### Risks
1. **High Volatility**: Nvidia's stock has shown high volatility with a 159.50% increase over the past six months, which poses a significant risk.
2. **Strong Fundamentals**: Nvidia has strong fundamental performance and a positive earnings outlook, which could counter the short thesis.
3. **Technical Breakthrough**: The risk of breaking through technical resistance levels could lead to significant losses.
4. **Positive Industry Trends**: The AI and semiconductor industries are trending positively, which could bolster Nvidia’s stock price.
5. **Leverage Risk**: Using 5x leverage increases the potential for magnified losses.

#### Execution Strategy
1. Enter short positions near resistance levels ($137.85, $144.93, $149.09).
2. Set stop losses slightly above resistance levels (e.g., $138.00).
3. Target profits at support levels ($126.61, $122.45, $115.37).
4. Continuously monitor market conditions and adjust the strategy as needed.
5. Use 5x leverage with a 2% risk per trade to manage potential losses.

### Mitigation Strategies
1. **Utilize Inverse ETPs**: Consider using inverse Exchange-Traded Products (ETPs) like Leverage Shares -1x Short Nvidia and -3x Short Nvidia ETPs. This approach caps potential losses to the amount invested.
2. **Derivatives for Hedging**: Employ derivatives such as options, warrants, futures, and Contracts for Difference (CFDs) to hedge against potential losses.
3. **Set Stop-Loss Orders**: Implement stop-loss orders slightly above resistance levels to limit losses.
4. **Leverage Management**: Continuously monitor and adjust leverage as needed to avoid magnified losses.
5. **Diversification**: Ensure the short position in Nvidia is part of a diversified portfolio to spread risk.
6. **Stay Informed on Industry Trends**: Regularly update on industry trends and Nvidia’s performance, especially in AI and semiconductor sectors.
7. **Consult Financial Advisors**: Seek advice from financial professionals to tailor risk management strategies to specific situations.

### Conclusion
By combining the insights from the Data Analyst, Trading Strategy Developer, Trade Advisor, and Risk Advisor, we can make a well-informed decision on shorting Nvidia's stock. The comprehensive risk analysis and mitigation strategies ensure that we are prepared to manage potential risks while aiming for maximum returns. Regular monitoring and adjustments will be crucial to adapt to changing market conditions and maintain a balanced risk-reward profile.

In [9]:
print(financial_trading_crew.usage_metrics)

{'total_tokens': 246002, 'prompt_tokens': 219907, 'completion_tokens': 26095, 'successful_requests': 57}
